In [1]:
import sys
import requests
import time
from bs4 import BeautifulSoup

import xlsxwriter

In [2]:
def escribimos_excel(in_lista_datos):
        print("       Instanciamos el ficehro de salida")        
        #Escribimos el fichero de salida
        workbook = xlsxwriter.Workbook("Datos_Motos_"+time.strftime("%d_%m_%Y")+".xlsx")
        worksheet = workbook.add_worksheet()
        # Start from the first cell. Rows and columns are zero indexed.
        row = 1
        col = 0
        worksheet.write(0, col,     "Modelo")
        worksheet.write(0, col + 1, "anio")
        worksheet.write(0, col + 2, "km")
        worksheet.write(0, col + 3, "provincia")
        worksheet.write(0, col + 4, "cilindrada")
        worksheet.write(0, col + 5, "url")
        worksheet.write(0, col + 6, "Preio")
        worksheet.write(0, col + 7, "fec_publicaion")
        worksheet.write(0, col + 8, "Comentario")
        worksheet.write(0, col + 9, "pagina")

        print("       Iniciamos el recorrido del Loop")
        
        # Iterate over the data and write it out row by row.
        for modelo,anio,km,provincia,cilindrada,url,preio,fec_publicaion,comentario,pagina in (in_lista_datos):
            worksheet.write(row, col,     modelo)
            worksheet.write(row, col + 1, int(anio))
            worksheet.write(row, col + 2, int("0"+km.replace(".","").replace(" km","").replace("N/D","").strip()))
            worksheet.write(row, col + 3, provincia)
            worksheet.write(row, col + 4, cilindrada)
            worksheet.write(row, col + 5, "http://motos.coches.net"+url)
            worksheet.write(row, col + 6, int("0"+preio.replace(".","").replace(" €","").replace("N/D","").strip()))
            worksheet.write(row, col + 7, fec_publicaion)
            worksheet.write(row, col + 8, comentario)
            worksheet.write(row, col + 9, pagina)

            row += 1
        num_filas = len(in_lista_datos)
        print("Guardando.... %d  filas" % num_filas)
        workbook.close()
        return "FIN"

In [3]:
def obtener_comentarios(dir_url):
    #print("      Obtenemos comentarios del anuncio")
    # spoof some headers so the request appears to be coming from a browser, not a bot
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
        "accept-encoding": "gzip,deflate,sdch",
        "accept-language": "en-US,en;q=0.8",
        }

    # make the request to the search url, passing in the the spoofed headers.
    r = requests.get(dir_url, headers=headers)  # assign the response to a variable r

    # check the status code of the response to make sure the request went well
    if r.status_code != 200:
        print("request denied")
        print(r.status_code)
        sigue = 0

    #else:
        #print("scraping " + url)  


    # convert the plaintext HTML markup into a DOM-like structure that we can search
    soup = BeautifulSoup(r.text, 'html.parser')# each result is an <li> element with class="g" this is our wrapper

    results = soup.findAll("div", attrs={"id":"div_comentarios"})

    #print(results)
    
    if len(results) == 0:
        #print("No hay comentarios")
        comentario = ""

    else:
        # iterate over each of the result wrapper elements
        comentario = results[0].findAll("p")[0].text.strip()
        
    #print("Comentario:  %s" % comentario)
    return comentario

In [6]:
def scrape_motos():

        pagina = 0  
        sigue = 1

        lista_datos = []

        while sigue == 1:
            
            pagina += 1
            print("Pagina: %s" % pagina)
          
            # dynamically build the URL that we'll be making a request to
            url = "http://motos.coches.net/ocasion/default.aspx?pg="+str(pagina)+"&fi=CreationDate"
            # spoof some headers so the request appears to be coming from a browser, not a bot
            headers = {
                "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_5)",
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "accept-charset": "ISO-8859-1,utf-8;q=0.7,*;q=0.3",
                "accept-encoding": "gzip,deflate,sdch",
                "accept-language": "en-US,en;q=0.8",
                }

            # make the request to the search url, passing in the the spoofed headers.
            r = requests.get(url, headers=headers)  # assign the response to a variable r

            # check the status code of the response to make sure the request went well
            if r.status_code != 200:
                print("request denied")
                print(r.status_code)
                sigue = 0

            #else:
                #print("scraping " + url)  


            # convert the plaintext HTML markup into a DOM-like structure that we can search
            soup = BeautifulSoup(r.text, 'html.parser')# each result is an <li> element with class="g" this is our wrapper

            results = soup.findAll("a", attrs={"class":"lnkad"})

            #Para pruebas
            #if len(results) == 0 or pagina == 2:
            
            if len(results) == 0:
                sigue = 0
                print("Paramos de recuperar páginas, la última no tiene datos")

            else:
                # iterate over each of the result wrapper elements
                for result in results:
                    modelo = result.attrs["title"]
                    url=result.attrs["href"]

                    preio = result.findAll("p", attrs={"class":"preu"})[0].text
                    fec_publicaion = result.findAll("p", attrs={"class":"data floatright"})[0].text

                    datos = result.findAll("p", attrs={"class":"dades"})[0]

                    #print(datos)
                    km = datos.findAll("span", attrs={"class":"d1"})[0].text
                    anio = datos.findAll("span", attrs={"class":"d2"})[0].text  
                    cilindrada = datos.findAll("span", attrs={"class":"d3"})[0].text
                    provincia = datos.findAll("span", attrs={"class":"lloc"})[0].text

                    comentario=obtener_comentarios("http://motos.coches.net"+url)
                    
                    
                    lista_datos.append([modelo,anio,km,provincia,cilindrada,url,preio,fec_publicaion,comentario,pagina])

        return lista_datos

In [7]:
print("Iniciamos Scrapeo")
scrape_datos = scrape_motos()
print("Fin Scrapeo e iniciamos escritura en fichero")
escribimos_excel(scrape_datos)
print("Fin de PROCESO")

Iniciamos Scrapeo
Pagina: 1
Pagina: 2
Pagina: 3
Pagina: 4
Pagina: 5
Pagina: 6
Pagina: 7
Pagina: 8
Pagina: 9
Pagina: 10
Pagina: 11
Pagina: 12
Pagina: 13
Pagina: 14
Pagina: 15
Pagina: 16
Pagina: 17
Pagina: 18
Pagina: 19
Pagina: 20
Pagina: 21
Pagina: 22
Pagina: 23
Pagina: 24
Pagina: 25
Pagina: 26
Pagina: 27
Pagina: 28
Pagina: 29
Pagina: 30
Pagina: 31
Pagina: 32
Pagina: 33
Pagina: 34
Pagina: 35
Pagina: 36
Pagina: 37
Pagina: 38
Pagina: 39
Pagina: 40
Pagina: 41
Pagina: 42
Pagina: 43
Pagina: 44
Pagina: 45
Pagina: 46
Pagina: 47
Pagina: 48
Pagina: 49
Pagina: 50
Pagina: 51
Pagina: 52
Pagina: 53
Pagina: 54
Pagina: 55
Pagina: 56
Pagina: 57
Pagina: 58
Pagina: 59
Pagina: 60
Pagina: 61
Pagina: 62
Pagina: 63
Pagina: 64
Pagina: 65
Pagina: 66
Pagina: 67
Pagina: 68
Pagina: 69
Pagina: 70
Pagina: 71
Pagina: 72
Pagina: 73
Pagina: 74
Pagina: 75
Pagina: 76
Pagina: 77
Pagina: 78
Pagina: 79
Pagina: 80
Pagina: 81
Pagina: 82
Pagina: 83
Pagina: 84
Pagina: 85
Pagina: 86
Pagina: 87
Pagina: 88
Pagina: 89
Pagina: 90
P